In [2]:
# 5.12.1 稠密块
# DenseNet使用了ResNet改良版的“批量归一化、激活和卷积”的卷积块结构，我们首先在conv_block函数里实现这个卷积块结构。
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import nn

def conv_block(num_channels):# num_channels为输出通道数
    blk=nn.Sequential()
    blk.add(nn.BatchNorm(),nn.Activation('relu'),# “批量归一化、激活和卷积”结构 
           nn.Conv2D(num_channels,kernel_size=3,padding=1))
    return blk

In [4]:
# 稠密块由num_convs个conv_block组成，每个卷积块使用相同的输出通道数。但在前向计算时，我们将每个卷积块的输入X和输出Y在通道维上连结。
class DenseBlock(nn.Block):
    def __init__(self,num_convs,num_channels,**kwargs):
        super(DenseBlock,self).__init__(**kwargs)
        self.net=nn.Sequential()
        for _ in range(num_convs):# num_convs为卷积块数
            self.net.add(conv_block(num_channels))# num_channels为输出通道数
            
    def forward(self,X):
        for blk in self.net:
            Y=blk(X)
            X=nd.concat(X,Y,dim=1)# 在通道维上将输入和输出连结
        return X

In [8]:
# 定义一个有2个输出通道数为10的卷积块。使用通道数为3的输入时，我们会得到通道数为 3+2×10=23 的输出。
# 卷积块的通道数控制了输出通道数相对于输入通道数的增长，因此也被称为增长率（growth rate）。
blk=DenseBlock(2,10)
blk.initialize()
X=nd.random.uniform(shape=(4,3,8,8))
Y=blk(X)
Y.shape

(4, 23, 8, 8)

In [9]:
# 5.12.2 过渡层 
# 由于每个稠密块都会带来通道数的增加，使用过多则会带来过于复杂的模型。过渡层用来控制模型复杂度。
# 它通过 1×1 卷积层来减小通道数，并使用步幅为2的平均池化层减半高和宽，从而进一步降低模型复杂度。
# 过渡层也使用“批量归一化、激活和卷积”的卷积块结构
def transition_block(num_channels):# 输出通道数为num_channels
    blk=nn.Sequential()
    blk.add(nn.BatchNorm(),nn.Activation('relu'),
           nn.Conv2D(num_channels,kernel_size=1),
           nn.AvgPool2D(pool_size=2,strides=2))
    return blk

In [12]:
blk=transition_block(10)
blk.initialize()
blk(Y).shape

(4, 10, 4, 4)

In [24]:
# 5.12.3 DenseNet模型
# DenseNet首先使用同ResNet一样的单卷积层和最大池化层。
net=nn.Sequential()
net.add(nn.Conv2D(64,kernel_size=7,strides=2,padding=3),
       nn.BatchNorm(),nn.Activation('relu'),
       nn.MaxPool2D(pool_size=3,strides=2,padding=1))

In [25]:
# DenseNet使用4个稠密块。可以设置每个稠密块使用多少个卷积层。
# 稠密块里的卷积层通道数（即增长率）设为32，所以每个稠密块将增加128个通道。
# ResNet里通过步幅为2的残差块在每个模块之间减小高和宽。这里我们则使用过渡层来减半高和宽，并减半通道数。
num_channels,growth_rate=64,32 # num_channels为当前的通道数
num_convs_in_dense_blocks=[4,4,4,4]# 每个稠密块的卷积层数。

for i,num_convs in enumerate(num_convs_in_dense_blocks):# enumerate获得(index,value)的迭代器
    net.add(DenseBlock(num_convs,growth_rate))
    # 上一个稠密块的输出通道数
    num_channels+=num_convs*growth_rate
    # 在稠密块之间加入通道数减半的过渡层
    if i !=len(num_convs_in_dense_blocks)-1:# 不是最后一个稠密块，则加入过渡层
        num_channels//=2
        net.add(transition_block(num_channels))        

In [26]:
# 同ResNet一样，最后接上全局池化层和全连接层来输出。
net.add(nn.BatchNorm(),nn.Activation('relu'),nn.GlobalAvgPool2D(),
       nn.Dense(10))

In [27]:
# 5.12.4 获取数据并训练模型 
import mxnet as mx
lr,num_epochs,batch_size,ctx=0.1,5,256,mx.cpu()
net.initialize(ctx=ctx,init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size,resize=96)
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on cpu(0)


KeyboardInterrupt: 

In [17]:
enumerate??